# Implementing Iteration

## Agenda

1. Review: Iteration
2. Details: *iterables*, *iterators*, `iter`, and `next`
3. Implementing iterators with classes
4. Implementing iterators with *generators* and `yield`

## 1. Iteration

*Iteration* simply refers to the process of accessing — one by one — the items stored in some container. The order of the items, and whether or not the iteration is comprehensive, depends on the container.

In Python, we typically perform iteration using the `for` loop.

In [1]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [2]:
for c in 'CS 331':
    print(c)

C
S
 
3
3
1


In [3]:
for e in [3.14, 'hello', True]:
  print(e)

3.14
hello
True


In [4]:
for n in {5, 6, 7, 5, 3, 0, 9}:
  print(n)

0
3
5
6
7
9


In [5]:
for k,v in {'a': 'aardvark', 'b': 'bear', 'c': 'cat'}.items():
    print(f'{k} -> {v}')

a -> aardvark
b -> bear
c -> cat


## 2. *iterables*, *iterators*, `iter`, and `next`

We can iterate over anything that is *iterable*. Intuitively, if something can be used as the source of items in a `for` loop, it is iterable.

But how does a `for` loop really work?

In [15]:
l = ['iteration', 'is', 'really', 'very', 'convenient']

In [16]:
it = iter(l)

In [8]:
type(l), type(it)

(list, list_iterator)

In [14]:
next(it)

StopIteration: 

In [17]:
# putting it all together
it = iter(l)
while True:
    x = next(it)
    print(x)

iteration
is
really
very
convenient


StopIteration: 

In [18]:
# iter & next are based on the __iter__ and __next__ special methods
it = l.__iter__()
while True:
    x = it.__next__()
    print(x)

iteration
is
really
very
convenient


StopIteration: 

## 3. Implementing iterators with classes

In [19]:
class MyIterator:
    def __init__(self, max):
        self.max = max
        self.curr = 0
        
    # the following methods are required for iterator objects

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.curr < self.max:
            ret = self.curr
            self.curr += 1
            return ret
        else:
            raise StopIteration

In [32]:
it = MyIterator(10)

In [33]:
next(it)

0

In [34]:
it = MyIterator(10)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

0
1
2
3
4
5
6
7
8
9


In [35]:
it = MyIterator(10)
for i in it:
    print(i)

0
1
2
3
4
5
6
7
8
9


An iterator is a *one time use object*! I.e., once we've used it to iterate over elements we cannot typically reset or "rewind" iteration. 

In [36]:
it = MyIterator(10)

In [41]:
# try running this cell multiple times
for i in it:
  print(i)

Iterable objects that can be traversed repeatedly return fresh iterators for each traversal.

In [45]:
# what is happening each time we iterate over `l`, below?

l = list(range(10))
total = 0
for o in range(10):
    print(o)
    for x in l:
        print(x)
        total += x
        print(total, '\n')
total

0
0
0 

1
1 

2
3 

3
6 

4
10 

5
15 

6
21 

7
28 

8
36 

9
45 

1
0
45 

1
46 

2
48 

3
51 

4
55 

5
60 

6
66 

7
73 

8
81 

9
90 

2
0
90 

1
91 

2
93 

3
96 

4
100 

5
105 

6
111 

7
118 

8
126 

9
135 

3
0
135 

1
136 

2
138 

3
141 

4
145 

5
150 

6
156 

7
163 

8
171 

9
180 

4
0
180 

1
181 

2
183 

3
186 

4
190 

5
195 

6
201 

7
208 

8
216 

9
225 

5
0
225 

1
226 

2
228 

3
231 

4
235 

5
240 

6
246 

7
253 

8
261 

9
270 

6
0
270 

1
271 

2
273 

3
276 

4
280 

5
285 

6
291 

7
298 

8
306 

9
315 

7
0
315 

1
316 

2
318 

3
321 

4
325 

5
330 

6
336 

7
343 

8
351 

9
360 

8
0
360 

1
361 

2
363 

3
366 

4
370 

5
375 

6
381 

7
388 

8
396 

9
405 

9
0
405 

1
406 

2
408 

3
411 

4
415 

5
420 

6
426 

7
433 

8
441 

9
450 



450

In [46]:
l = list(range(10))
total = 0
for _ in range(10):
    it = iter(l) # we obtain and "use up" a new iterator each loop!
    while True:
        try:
            total += next(it)
        except StopIteration:
            break
total

450

For a container type, we need to implement an `__iter__` method that returns an iterator.

In [47]:
import numpy as np

class ArrayListIterator:
    def __init__(self, array_list):
        self.array_list = array_list
        self.curr_idx = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.curr_idx < self.array_list.size:
            ret = self.array_list.data[self.curr_idx]
            self.curr_idx += 1
            return ret
        else:
            raise StopIteration


class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0
        
    def append(self, val):
        if self.size == len(self.data):
            ndata = np.empty(2 * len(self.data), dtype=object)
            for i in range(len(self.data)):
                ndata[i] = self.data[i]
            self.data = ndata

        self.data[self.size] = val
        self.size += 1
        
    def __iter__(self):
        return ArrayListIterator(self) # create and return an iterator

In [48]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [49]:
it = iter(l)

In [50]:
type(it)

__main__.ArrayListIterator

In [62]:
next(it)

StopIteration: 

In [52]:
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


## 4. Implementing iterators with generators

What's a "generator"?

In [63]:
l = [2**x for x in range(10)]
g = (2**x for x in range(10))

In [64]:
type(l), type(g)

(list, generator)

In [67]:
# try running this cell repeatedly
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


In [74]:
# try running this cell repeatedly
for x in g:
    print(x)

In [75]:
dir(g)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [78]:
help(g.throw)
help(g.send)

Help on built-in function throw:

throw(...) method of builtins.generator instance
    throw(typ[,val[,tb]]) -> raise exception in generator,
    return next yielded value or raise StopIteration.

Help on built-in function send:

send(...) method of builtins.generator instance
    send(arg) -> send 'arg' into generator,
    return next yielded value or raise StopIteration.



Note: Generators implement the iterator API! (`__iter__` and `__next__`)

In [79]:
g = (2**x for x in range(10))

In [80]:
g is iter(g)

True

In [91]:
next(g)

StopIteration: 

In [92]:
%timeit -n 1000 [2**x for x in range(1000)]

1.14 ms ± 90.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [93]:
%timeit -n 1000 (2**x for x in range(1000))

764 ns ± 39.6 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [94]:
g = (2**x for x in range(100))

In [95]:
g[100]

TypeError: 'generator' object is not subscriptable

In [102]:
# try running this cell repeatedly
sum(g)

0

A *generator expression* syntactically resembles a list comprehension, and is similar in that it evaluates to an iterable sequence of values. However, a generator does not represent a fully fleshed out collection of values; instead, values are returned only as they are required through the iteration API (i.e., `next`) --- we refer to this as *lazy evaluation*. 

This makes a generator more efficient than a list (since we don't need to keep all values in the sequence around), but generators can't replace lists in all scenarios (e.g., when we need to jump around in the sequence or revisit values).

### Creating generator functions: `yield`

In [103]:
def foo():
    n = 0
    while True:
        yield 2**n
        n += 1

In [104]:
foo()

<generator object foo at 0x7f001dd3a3c0>

In [105]:
type(foo())

generator

In [107]:
f = foo()
next(f)

1

In [346]:
next(f)

883423532389192164791648750371459257913741948437809479060803100646309888

In [347]:
def foo():
    print('hello!')
    yield
    print('goodbye!')

In [348]:
foo()

<generator object foo at 0x7f001e61c900>

In [349]:
g = foo()

In [351]:
next(g)

goodbye!


StopIteration: 

In [352]:
def foo():
    yield 1
    yield 2
    yield 3

In [353]:
g = foo()

In [357]:
next(g)

StopIteration: 

In [358]:
def countdown(n):
    for x in range(n, 0, -1):
        yield x
    yield 'Blast off!'

In [359]:
for x in countdown(5):
    print(x)

5
4
3
2
1
Blast off!


In [360]:
list(countdown(10))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 'Blast off!']

A *generator function* is a function that contains one or more `yield` statements. When called, a generator function returns a generator object, which effectively allows us to incrementally execute the function using the iteration API. Each call to `next` on the generator will execute the function up to the next `yield` statement; if/when the function completes the generator will raise a `StopIteration` exception (just like an iterator).

### Generators as Data Structure Iterators

In [374]:
class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0
        
    def append(self, val):
        if self.size == len(self.data):
            ndata = np.empty(len(self.data)*2, dtype=object)
            for i in range(len(self.data)):
                ndata[i] = self.data[i]
            self.data = ndata

        self.data[self.size] = val
        self.size += 1
        
    def __iter__(self):
        next_idx = 0
        while True:
            if next_idx <= self.size - 1:
                yield self.data[next_idx]
                next_idx += 1
            else:
                break
                
                
    def __repr__(self):
        """Supports inspection"""
        repr_elems = [repr(self.data[i]) for i in range(self.size)]
        return '[' + ', '.join(repr_elems) + ']'

In [375]:
l = ArrayList()
for x in range(10):
    l.append(2**x)
    
l

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]

In [376]:
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


In [ ]:
# use the now functioning iterator to implement __repr__
class ArrayList(ArrayList):
    def __repr__(self):
        pass

In [ ]:
l = ArrayList()
for x in range(10):
    l.append(2**x)
l